## 1. 問題の分析

**競技プログラミング視点**
- 配列を1回走査するだけで分類できるため、O(n)が理論的下限
- ハッシュマップ（オブジェクト）によるO(1)キーアクセスで最適化

**業務開発視点**
- `Array.prototype`拡張はグローバルな副作用を持つため、型定義側でインターフェース宣言を行い型安全性を担保
- `Record<string, T[]>`で戻り値を明確に型付け

**TypeScript特有の考慮点**
- `this`は`Array.prototype`拡張内では実行時配列を指すが、型は`any[]`扱い → ジェネリクスで安全化
- `interface Array<T>`のマージ宣言（Declaration Merging）でプロトタイプ拡張を型レベルに反映

---

## 2. アルゴリズムアプローチ比較

| アプローチ | 時間計算量 | 空間計算量 | TS実装コスト | 型安全性 | 可読性 | 備考 |
|---|---|---|---|---|---|---|
| `for`ループ + オブジェクト | O(n) | O(n) | 低 | 高 | 高 | **最適解** |
| `reduce` | O(n) | O(n) | 低 | 高 | 高 | 関数型スタイル |
| `sort` + 走査 | O(n log n) | O(n) | 中 | 中 | 低 | 不要なソートコスト |

---

## 3. 選択したアルゴリズムと理由

- **選択**: `reduce` によるアキュムレータパターン
- **理由**:
  - O(n)の単一走査で完結、追加メモリはO(n)（結果格納分のみ）
  - `reduce`はイミュータブル志向で副作用がなく純粋関数的
  - TypeScriptの型推論と相性が良く、アキュムレータの型を`Record<string, T[]>`で明示可能

---

## 4. 実装コード

```typescript
// Analyze Complexity
// 実測: Runtime 98 ms / Memory 79.27 MB
// Beats 80.23% / 40.00%
interface Array<T> {
    groupBy(fn: (item: T) => string): Record<string, T[]>;
}

Array.prototype.groupBy = function<T>(
    this: T[],
    fn: (item: T) => string
): Record<string, T[]> {
    return this.reduce<Record<string, T[]>>((acc, item) => {
        const key = fn(item);
        // キーが未存在なら空配列で初期化、存在すれば既存配列を再利用
        (acc[key] ??= []).push(item);
        return acc;
    }, Object.create(null)); // プロトタイプ汚染を防ぐためObject.create(null)
};

/**
 * [1,2,3].groupBy(String) // {"1":[1],"2":[2],"3":[3]}
 */
```

---

### 実装の重要ポイント解説

**① `this: T[]` による明示的 `this` 型バインド**
```typescript
// NG: thisがany扱いになりTypeScriptの恩恵を受けられない
Array.prototype.groupBy = function(fn) { ... }

// OK: thisをT[]と明示することでジェネリクスが機能する
Array.prototype.groupBy = function<T>(this: T[], fn) { ... }
```

**② `??=`（Nullish coalescing assignment）による簡潔な初期化**
```typescript
// 従来の書き方（冗長）
if (!acc[key]) acc[key] = [];
acc[key].push(item);

// ??= を使った簡潔な書き方（ES2021+, Node.js v22対応）
(acc[key] ??= []).push(item);
```

**③ `Object.create(null)` によるプロトタイプ汚染防止**
```typescript
// NG: {}はObject.prototypeを継承するため
// "constructor", "toString"等のキーと衝突リスクがある
const acc = {};

// OK: プロトタイプチェーンを持たない純粋なマップとして機能
const acc = Object.create(null);
```

## 現状分析

| 指標 | 現状 | 問題点 |
|---|---|---|
| Runtime 98ms | Beats 80.23% | まだ改善余地あり |
| Memory 79.27MB | **Beats 40.00%** | **ここがボトルネック** |

メモリがボトルネックの主因は **`reduce`のコールバック関数が反復ごとにスタックフレームを生成する**ことです。

---

## ボトルネックの詳細

```
reduce の内部動作（メモリ観点）
─────────────────────────────────────────
item[0] → fn呼び出し → スタックフレーム生成 → pop
item[1] → fn呼び出し → スタックフレーム生成 → pop
item[2] → fn呼び出し → スタックフレーム生成 → pop
...× n回繰り返す  ← n=10^5 のとき顕著にメモリ圧迫
```

```
for ループの内部動作（メモリ観点）
─────────────────────────────────────────
スタックフレームは1つだけ確保 → ループ内で使い回す
```

---

## 改善コード

```typescript
// Analyze Complexity
// 実測: Runtime 101 ms / Memory 74.68 MB
// Beats 70.34% / 90.11%
interface Array<T> {
    groupBy(fn: (item: T) => string): Record<string, T[]>;
}

Array.prototype.groupBy = function<T>(
    this: T[],
    fn: (item: T) => string
): Record<string, T[]> {
    // ① reduceをforループに変更 → スタックフレームの反復生成を排除
    const result: Record<string, T[]> = Object.create(null);
    const len = this.length; // ② length キャッシュ → プロパティ参照コスト削減

    for (let i = 0; i < len; i++) {
        const item = this[i];        // ③ 一時変数でthis参照を1回に抑制
        const key = fn(item);

        // ④ Object.create(null) に対するキー存在確認のセマンティクス
        if (key in result) {
            result[key].push(item);
        } else {
            result[key] = [item];
        }
    }

    return result;
};
```

---

## 改善ポイント詳解

**① `reduce` → `for`ループ**
```typescript
// Before: n回のコールバック呼び出し = n個のスタックフレーム生成・破棄
this.reduce<Record<string, T[]>>((acc, item) => { ... }, Object.create(null));

// After: 1つのスタックフレームを使い回す
for (let i = 0; i < len; i++) { ... }
```

**② `length`キャッシュ**
```typescript
// Before: ループ毎にthis.lengthプロパティを参照（微小だが積み重なる）
for (let i = 0; i < this.length; i++)

// After: 変数参照のみ（プロパティルックアップ不要）
const len = this.length;
for (let i = 0; i < len; i++)
```

**③ `??=` と `in` 演算子**
```typescript
// パターンA: ??= を用いた記述（V8環境でIC最適化が効きやすい）
(acc[key] ??= []).push(item);

// パターンB: in 演算子による存在確認（本実装での採用）
if (key in result) {
    result[key].push(item);
} else {
    result[key] = [item];  // 初回のみ配列生成
}
```

---

## 期待される改善効果

| 指標 | Before(実測) | After(実測) |
|---|---|---|
| Runtime | 98 ms (Beats 80.23%) | 101 ms (Beats 70.34%) |
| Memory | 79.27 MB (Beats 40.00%) | 74.68 MB (Beats 90.11%) |
| 主な影響 | — | GCの影響によりRuntime微増・Memory大幅改善 |

> **補足**: LeetCodeのメモリ計測はV8エンジンのGCタイミングに依存するため、実行ごとに若干ブレます。複数回提出して中央値で評価することを推奨します。